## Búsqueda de ubicación óptima para instalaciones

### Importación de librerías y módulos

In [31]:
from pymongo import MongoClient
from src.coordenadas import *
from src.cleaning import *
from src.airports import *
from src.representation import *
from src.init_tokens import *
from src.distanceCalc import *
from src.reverseGeocoding import *
import random
import pandas as pd

import geopandas as gpd
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget

### Necesidades

**Diseñadores** ➡️ Compañías de diseño

**Empleados con hijos** ➡️ Guardería

**Desarrolladores** ➡️ Empresas de tecnología de éxito

**Ejecutivos** ➡️ Establecimiento de Starbucks

**Contables** ➡️ Aeropuerto

**Consenso** ➡️ Empresas < 10 años

**Empleados jóvenes** ➡️ Sitios de fiesta

**CEO** ➡️ Restaurantes veganos


### Elección de prioridades

**Modelo aleatorio para narrativa**

In [64]:
necesidades = ["Diseñadores","Empleados con hijos","Desarrolladores","Ejecutivos","Contables","Empleados jóvenes","Todos","CEO"]

random_priorities = list(random.sample(range(1,len(necesidades)+1),len(necesidades)))

prioridades = sorted(list(map(lambda x: (random_priorities[necesidades.index(x)],x),necesidades)))[:3]

prioridades


[(1, 'Ejecutivos'), (2, 'Diseñadores'), (3, 'Empleados con hijos')]

### Preparación de triangulación mundial

**Se procede a ubicar a aquellas empresas relacionadas con el diseño y aquellas tecnológicas con un volumen de negocio superior a los 10 millones de dólares**

**Importación de Dataset de Compañías a nivel mundial**

In [33]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/AnalysisPlaces")

db = client.get_database()

db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'AnalysisPlaces')

**Corrección de coordenadas para GeoJson para aquellas compañías que siguen en activo**

In [34]:

officesCoord = cleaning(db["companiesOriginal"])

# officesCoord.to_json("Outputs/companiesCoord.json", orient="records")

officesCoord.head(3)


,name,category_code,founded_year,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,location
0,Geni,web,2006,$16.5M,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,"{'type': 'Point', 'coordinates': [34.090368, -..."
1,Digg,news,2004,$45M,None,135 Mississippi St,,94107,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [37.764726, -..."
2,Gizmoz,web,2003,$18.1M,None,None,None,None,Menlo Park,CA,USA,"{'type': 'Point', 'coordinates': [37.48413, -1..."


**Creación de DataFrame con aquellas compañías que posean una localización válida**

In [35]:
filter = {"$and":[{"location":{"$not":{"$size":0}}},{"location":{"$ne":None}},{"location":{"$exists":True}}]}
project={"name":1,"offices":1,"category_code":1, "founded_year":1, "total_money_raised":1, "location":1,"_id":0}

test = pd.DataFrame(list(db['companiesCoord'].find(filter=filter,projection=project)))

test.head(3)

,name,category_code,founded_year,total_money_raised,location
0,Helio,mobile,2005,$0,"{'type': 'Point', 'coordinates': [34.057498, -..."
1,Geni,web,2006,$16.5M,"{'type': 'Point', 'coordinates': [34.090368, -..."
2,StumbleUpon,web,2002,$18.5M,"{'type': 'Point', 'coordinates': [37.775196, -..."


**Creación de columna de coordenadas principales en Dataframe anterior**

In [36]:
dfCoords = test[["location"]].apply(lambda r: r.location, result_type="expand", axis=1)

dfCoords.head(3)

,type,coordinates
0,Point,"[34.057498, -118.446596]"
1,Point,"[34.090368, -118.393064]"
2,Point,"[37.775196, -122.419204]"


In [37]:
test["long"] = dfCoords["coordinates"].apply(lambda x: x[0])
test["lat"] = dfCoords["coordinates"].apply(lambda x: x[1])

test = test[['name', 'category_code', 'founded_year', 'total_money_raised', 'long', 'lat', 'location']] 

test.head(3)

,name,category_code,founded_year,total_money_raised,long,lat,location
0,Helio,mobile,2005,$0,34.057498,-118.446596,"{'type': 'Point', 'coordinates': [34.057498, -..."
1,Geni,web,2006,$16.5M,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [34.090368, -..."
2,StumbleUpon,web,2002,$18.5M,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [37.775196, -..."


**Búsqueda de empresas de diseño**

In [38]:
Listado_Categorias = test.groupby('category_code').count().reset_index()

Listado_Categorias["category_code"].to_list()

['advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

In [39]:
designCompanies = test.copy()

designCompanies["category_code"] = designCompanies["category_code"].apply(cleaning_design)

designCompanies = designCompanies.dropna()

designCompanies.head(3)



,name,category_code,founded_year,total_money_raised,long,lat,location
1,Geni,web,2006,$16.5M,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [34.090368, -..."
2,StumbleUpon,web,2002,$18.5M,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [37.775196, -..."
6,Fox Interactive Media,web,1979,$0,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [34.076179, -..."


**Búsqueda de empresas de tecnología de éxito (Más de 10 millones de dólares)**

In [40]:
techCompanies = test.copy()

techCompanies["total_money_raised"] = techCompanies["total_money_raised"].apply(cleaning_money)

techCompanies["category_code"] = techCompanies["category_code"].apply(cleaning_tech)

techCompanies = techCompanies.dropna().loc[techCompanies['total_money_raised'] > 10]

techCompanies.head(3)

,name,category_code,founded_year,total_money_raised,long,lat,location
24,Jingle Networks,mobile,2005,88.7,37.480999,-122.173887,"{'type': 'Point', 'coordinates': [37.480999, -..."
37,Jajah,mobile,2005,33.0,37.423390,-122.089951,"{'type': 'Point', 'coordinates': [37.4233896, ..."
45,Skype,software,2003,76.8,51.518132,-0.131916,"{'type': 'Point', 'coordinates': [51.518132, -..."


### Preparación Starbucks

**Importación de Dataset con todos los establecimientos de Starbucks a nivel mundial**

In [41]:
starbucksLocation = pd.read_csv("Inputs/Starbucks_Locations.csv")

starbucksLocation.head(3)

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39


**Preparación de coordenadas y localización GeoJson de los locales de Starbucks**

In [42]:
starbucksLocation = starbucksLocation[["Store Name", "City", "Latitude", "Longitude"]].rename(columns={"Latitude":"lat","Longitude":"long"})

starbucksCoords = geojsonize(starbucksLocation)

starbucksCoords.head(3)

,Store Name,City,lat,long,location
0,"Meritxell, 96",Andorra la Vella,42.51,1.53,"{'type': 'Point', 'coordinates': [1.53, 42.51]}"
1,Ajman Drive Thru,Ajman,25.42,55.47,"{'type': 'Point', 'coordinates': [55.47, 25.42]}"
2,Dana Mall,Ajman,25.39,55.47,"{'type': 'Point', 'coordinates': [55.47, 25.39]}"


**Limpieza de fila errónea**

In [43]:
starbucksCoords.drop(index=[8028],inplace=True)

### Preparación Aeropuertos 

**Llamamiento de API con información de aeropuertos a nivel mundial**

In [44]:
airportsCoord = airportsDF()

airportsCoord.head()

,nombre,long,lat
0,Aleknagik / New Airport,-158.617996,59.282600
1,Honiara International Airport,160.054993,-9.428000
2,Munda Airport,157.263000,-8.327970
3,Mar de Cortés International Airport,-113.305864,31.351621
4,Nauru International Airport,166.919006,-0.547458


**Nueva columna con coordenadas en formato GeoJson**

In [45]:
airportsMap = geojsonize(airportsCoord)

airportsMap.head(3)

,nombre,long,lat,location
0,Aleknagik / New Airport,-158.617996,59.28260,"{'type': 'Point', 'coordinates': [-158.6179962..."
1,Honiara International Airport,160.054993,-9.42800,"{'type': 'Point', 'coordinates': [160.05499267..."
2,Munda Airport,157.263000,-8.32797,"{'type': 'Point', 'coordinates': [157.26300048..."


### Preparación Guarderías

**Comprobación de funcionamiento de API de Foursquare**

In [46]:
#IT WORKS!!

# init_four(lat, long)

## Ubicación de mejores coordenadas

**Centralización de Empresas en tres ubicaciones principales**

**Concatenación y distancia geográfica entre Empresas de Diseño y Empresas tecnológicas de éxito**

In [47]:
EmpresasDiseño = designCompanies.reset_index(drop=True)

EmpresasDiseño = EmpresasDiseño.rename(columns={"lat": "latd", "long": "longd", 'location':'locationd'})

EmpresasTechExito = techCompanies.reset_index(drop=True)


result = pd.concat([EmpresasTechExito,EmpresasDiseño], axis=1, sort=False)

result['Distance'] = result.apply(lambda x: distCalc(x['lat'], x['long'], x['latd'], x['longd']), axis=1)

result = result.sort_values(by='Distance', ascending=True).iloc[:10].reset_index(drop=True)

result[['lat', 'long', 'Distance', 'location']]

,lat,long,Distance,location
0,-122.402028,37.789129,0.258873,"{'type': 'Point', 'coordinates': [37.7891288, ..."
1,-122.397830,37.787126,0.469389,"{'type': 'Point', 'coordinates': [37.787126, -..."
2,-122.404107,37.787131,0.476385,"{'type': 'Point', 'coordinates': [37.7871306, ..."
3,-122.394354,37.782809,0.491888,"{'type': 'Point', 'coordinates': [37.782809, -..."
4,-73.978695,40.752113,0.778368,"{'type': 'Point', 'coordinates': [40.7521126, ..."
5,-73.993158,40.739834,1.016196,"{'type': 'Point', 'coordinates': [40.7398335, ..."
6,-122.419204,37.775196,1.209420,"{'type': 'Point', 'coordinates': [37.775196, -..."
7,-73.980305,40.751478,1.309565,"{'type': 'Point', 'coordinates': [40.751478, -..."
8,-122.257331,37.535876,1.393645,"{'type': 'Point', 'coordinates': [37.535876, -..."
9,-73.970624,40.761582,1.460472,"{'type': 'Point', 'coordinates': [40.7615818, ..."


**Reverse Geocoding!! Se procede a detallar las coordenadas de aquellas empresas más próximas entre sí**

In [48]:
result['reGeo'] = result.apply(lambda x: reverseGeocoding(x['long'], x['lat']), axis=1)

result = result[['lat', 'long', 'Distance', 'location', 'reGeo']]

result

,lat,long,Distance,location,reGeo
0,-122.402028,37.789129,0.258873,"{'type': 'Point', 'coordinates': [37.7891288, ...",San Francisco
1,-122.397830,37.787126,0.469389,"{'type': 'Point', 'coordinates': [37.787126, -...",San Francisco
2,-122.404107,37.787131,0.476385,"{'type': 'Point', 'coordinates': [37.7871306, ...",San Francisco
3,-122.394354,37.782809,0.491888,"{'type': 'Point', 'coordinates': [37.782809, -...",San Francisco
4,-73.978695,40.752113,0.778368,"{'type': 'Point', 'coordinates': [40.7521126, ...",New York
5,-73.993158,40.739834,1.016196,"{'type': 'Point', 'coordinates': [40.7398335, ...",New York
6,-122.419204,37.775196,1.209420,"{'type': 'Point', 'coordinates': [37.775196, -...",San Francisco
7,-73.980305,40.751478,1.309565,"{'type': 'Point', 'coordinates': [40.751478, -...",New York
8,-122.257331,37.535876,1.393645,"{'type': 'Point', 'coordinates': [37.535876, -...",Redwood City
9,-73.970624,40.761582,1.460472,"{'type': 'Point', 'coordinates': [40.7615818, ...",New York


**Obtención de tres ubicaciones con la menor distancia entre las Empresas de Diseño y las Empresas tecnológicas de éxito**

In [49]:
result =result.groupby('reGeo').min().sort_values(by='Distance', ascending=True).reset_index()
result


,reGeo,lat,long,Distance
0,San Francisco,-122.419204,37.775196,0.258873
1,New York,-73.993158,40.739834,0.778368
2,Redwood City,-122.257331,37.535876,1.393645


**Se procede a la homogeneización del resultado final con el resto de dataframes**

In [50]:
main = result.iloc[1:2]

main = main.rename(columns={"reGeo": "nombre","long":"lat","lat":"long"})

main = geojsonize(main)

main = main[["nombre","long","lat","location","Distance"]]

main

,nombre,long,lat,location,Distance
1,New York,-73.993158,40.739834,"{'type': 'Point', 'coordinates': [-73.9931582,...",0.778368


**El factor Starbucks**

**Se procede a buscar los Starbucks más cercanos a las mejores coordenadas entre empresas del apartado anterior**

In [51]:
Starbucks = starbucksCoords.copy()

Starbucks['reGeo'] = Starbucks.apply(lambda x: fixCoords(x['long'], x['lat']), axis=1).dropna()

Starbucks = Starbucks.dropna().reset_index(drop=True)

Starbucks = Starbucks[['Store Name','City','lat', 'long','location']]

Starbucks.head()


,Store Name,City,lat,long,location
0,Safeway - Alameda 3281,Alameda,37.79,-122.28,"{'type': 'Point', 'coordinates': [-122.28, 37...."
1,Park & Central Alameda,Alameda,37.76,-122.24,"{'type': 'Point', 'coordinates': [-122.24, 37...."
2,Webster & Atlantic - Alameda,Alameda,37.78,-122.28,"{'type': 'Point', 'coordinates': [-122.28, 37...."
3,Safeway - Alameda #2708,Alameda,37.76,-122.25,"{'type': 'Point', 'coordinates': [-122.25, 37...."
4,Tilden & Blanding,Alameda,37.77,-122.23,"{'type': 'Point', 'coordinates': [-122.23, 37...."


**Se localiza el local de Starbucks más cercano según la información de apartados anteriores**

In [52]:
# latDef = result["lat"].to_list()

# longDef = result["long"].to_list()

# coordsDef = list(zip(longDef,latDef))

Starbucks["DistanceStarbucks"] = Starbucks.apply(lambda x: compareCoords(x['long'], x['lat']), axis=1)

Starbucks = Starbucks.sort_values(by='DistanceStarbucks', ascending=True).iloc[:1]

Starbucks


,Store Name,City,lat,long,location,DistanceStarbucks
676,Union Square East,New York,40.74,-73.99,"{'type': 'Point', 'coordinates': [-73.99, 40.74]}",0.266806


**Se procede a la homogeneización del resultado final con el resto de dataframes**

In [53]:

Starbucks = Starbucks.rename(columns={"Store Name": "nombre", "DistanceStarbucks": "Distance"})

Starbucks = Starbucks[["nombre","long","lat","location","Distance"]]

Starbucks

,nombre,long,lat,location,Distance
676,Union Square East,-73.99,40.74,"{'type': 'Point', 'coordinates': [-73.99, 40.74]}",0.266806


**Una vez obtenida una coordenada que posee una Empresa tecnológica de éxito, una Empresa de diseño y un Starbucks a menos de 500 metros en la Ciudad de Nueva York, se procede a ubicar el aeropuerto más cercano a dicha coordenada**

In [54]:
Airports = airportsMap

Airports.head()


,nombre,long,lat,location
0,Aleknagik / New Airport,-158.617996,59.282600,"{'type': 'Point', 'coordinates': [-158.6179962..."
1,Honiara International Airport,160.054993,-9.428000,"{'type': 'Point', 'coordinates': [160.05499267..."
2,Munda Airport,157.263000,-8.327970,"{'type': 'Point', 'coordinates': [157.26300048..."
3,Mar de Cortés International Airport,-113.305864,31.351621,"{'type': 'Point', 'coordinates': [-113.305864,..."
4,Nauru International Airport,166.919006,-0.547458,"{'type': 'Point', 'coordinates': [166.919006, ..."


**Se localiza el aeropuerto más próximo a la coordenada principal**

In [55]:
Airports["Distance"] = Airports.apply(lambda x: distCalc(x['long'], x['lat'], -73.993158, 40.739834), axis=1)

Airports = Airports.sort_values(by='Distance', ascending=True).iloc[:10].reset_index(drop=True)

Airports = Airports.iloc[0:1]

Airports

	


,nombre,long,lat,location,Distance
0,La Guardia Airport,-73.872597,40.777199,"{'type': 'Point', 'coordinates': [-73.87259674...",13.459249


### El Factor Guardería

**A través de la API de Foursquare, se busca la guardería más cercana a la coordenada principal en la ciudad de Nueva York**

In [56]:
guarderias = init_four(40.739834, -73.993158)

https://api.foursquare.com/v2/venues/search?ll=40.739834,-73.993158&client_id=CF3DB20G304YY24ALQODEUIHOHLKLPIE2CWPGM1Z5ZFCRNJQ&client_secret=XA1BQCYJ4QN2ZLGID4FFQSOERYJKRFNAIV25G44Z4NC1H2Z5&v=20200204&categoryId=4f4532974b9074f6e4fb0104
200


**Se localiza la guardería más cercana a la coordenada en un radio inferior a 500 metros**

In [57]:
guarderiasTot = iteration(guarderias)

guarder = pd.DataFrame(guarderiasTot)

guarder = guarder.rename(columns={0:"nombre",1:"long",2:"lat",3:"Distance"})

guarder = guarder.sort_values(by='Distance', ascending=True).iloc[:10].reset_index(drop=True)

guarder = guarder[0:1]

guarder = geojsonize(guarder)

guarder = guarder[["nombre","long","lat","location","Distance"]]

guarder



,nombre,long,lat,location,Distance
0,Apple Seeds,-73.989614,40.743248,"{'type': 'Point', 'coordinates': [-73.98961352...",0.483


### Representación Gráfica

**Se concatenan todos los Dataframes resultantes en un Dataframe Final**

In [58]:
main

Airports

Starbucks

guarder

frames = [main,Airports,Starbucks,guarder]

resultDef = pd.concat(frames)

resultDef = resultDef.reset_index(drop=True)

resultDef = resultDef.rename(index={0:"Principal", 1:"Airport", 2:"Starbucks", 3:"Guarder"})

resultDef



,nombre,long,lat,location,Distance
Principal,New York,-73.993158,40.739834,"{'type': 'Point', 'coordinates': [-73.9931582,...",0.778368
Airport,La Guardia Airport,-73.872597,40.777199,"{'type': 'Point', 'coordinates': [-73.87259674...",13.459249
Starbucks,Union Square East,-73.990000,40.740000,"{'type': 'Point', 'coordinates': [-73.99, 40.74]}",0.266806
Guarder,Apple Seeds,-73.989614,40.743248,"{'type': 'Point', 'coordinates': [-73.98961352...",0.483000


**Se procede a mostrar de forma gráfica los puntos obtenidos anteriormente**

In [59]:
Map(Layer(resultDef))

**La calle elegida es "West 22nd St.", cerca del emblemático "Flatiron Building".**

### Resumen

Tras una triangulación mundial entre las empresas de diseño,las empresas tecnológicas con más de 10 millones de dólares en recaudación y los locales de Starbucks, se ha ubicado a Nueva York como el emplazamiento idóneo para las nuevas instalaciones de la compañía.

Una vez ubicada la ciudad principal, se ha procedido a localizar servicios destacados como guarderías y aeropuertos desde las coordenadas principales.